## Clase 8: Métodos de ensamblaje

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import linear_model as lm
import patsy

### Ejemplo Bagging

Vamos a ajustar un modelo sencillo con bagging aplicado a una regresión logística a modo de ejemplo.

In [ ]:
loan = pd.read_csv('loan.csv')

Partición train-test

In [ ]:
split = train_test_split(loan.index, test_size = 0.3)
train = loan.loc[split[0]]
test = loan.loc[split[1]]

Ajustar una regresión logística simple a modo de ejemplo. esta vez lo haremos con la librería sklearn

In [ ]:
ytr, Xtr = patsy.dmatrices("""SeriousDlqin2yrs ~ RevolvingUtilizationOfUnsecuredLines + DebtRatio +
                   np.log(MonthlyIncome + 1)""",
                  data = train)
yte, Xte = patsy.dmatrices("""SeriousDlqin2yrs ~ RevolvingUtilizationOfUnsecuredLines + DebtRatio +
                   np.log(MonthlyIncome + 1)""",
                   data = test)

In [ ]:
ft = lm.LogisticRegression()

Evaluemos los resultados

In [ ]:
ft = ft.fit(Xtr, np.ravel(ytr))
preds = ft.predict(Xte)

Ahora evaluemos resultados

In [ ]:
metrics.confusion_matrix(yte, preds)

Siempre recordar que el desbalance puede afectar en gran medida el resultado del modelo.

Una opción es el undersampling, es decir reducir la muestra hasta tener equilibrio entre las dos clases

In [ ]:
train_1 = train[train.SeriousDlqin2yrs == 1]
n1 = train_1.shape[0]
train_0 = train[train.SeriousDlqin2yrs == 0].sample(n1)
train_bal = pd.concat([train_1, train_0])

ytr, Xtr = patsy.dmatrices("""SeriousDlqin2yrs ~ RevolvingUtilizationOfUnsecuredLines + DebtRatio +
                   np.log(MonthlyIncome + 1)""",
                  data = train_bal)
ft = lm.LogisticRegression()
ft = ft.fit(Xtr, np.ravel(ytr))
preds = ft.predict(Xte)
metrics.confusion_matrix(yte, preds)

Generemos una función que haga un ajuste a partir de un muestreo

In [ ]:
def logistic(df, Xte, ft):
    ytr, Xtr = patsy.dmatrices("""SeriousDlqin2yrs ~ RevolvingUtilizationOfUnsecuredLines + DebtRatio +
                   np.log(MonthlyIncome + 1)""",
                  data = df)
    preds = ft.fit(Xtr, np.ravel(ytr)).predict(Xte)
    return preds

Generamos el bootstrap

In [ ]:
M = np.zeros((100, Xte.shape[0]))
n = train_bal.shape[0]
ft = lm.LogisticRegression()
      
for i in range(100):
    df = train_bal.sample(n, replace = True)
    M[i,:] = logistic(df, Xte, ft)

Se agregan los resultados de acuerdo a la clase mayoitaria en cada columna

In [ ]:
M

In [ ]:
preds = (M.sum(axis = 0) > 50) * 1

In [ ]:
preds

In [ ]:
metrics.confusion_matrix(yte, preds)

Nos aumentaron levemente los falsos negativos, pero disminuyeron notoriamente los falsos positivos.

### Ejemplo Random Forest

Primero, vamos a separar conjunto de train y test, la clase pasada lo hicimos manual, pero existen funciones para hacer esto

In [ ]:
from sklearn.ensemble import RandomForestClassifier

La función tiene algunos de los siguientes parámetros básicos.

```RandomForestClassifier(n_estimators=10, criterion=’gini’,max_features = 'auto', max_depth=None, min_samples_split=2, min_samples_leaf=1,  max_leaf_nodes=None, n_jobs=1, class_weight=None)```

- **n_estimators:** indica la cantidad de árboles a generar.
- **criterion:** indica el criterio para el ajuste de árboles.
- **max_depth:** profundida máxima de cada árbol
- **max_features:** Corresponde al $m$, cuantas variables se van a tomar al azar para la partición.
- **min_samples_split:** minima cantidad de muestrar para una división.
- **max_leaf_nodes:** máxima cantidad de nodos en el árbol final.
- **n_jobs:** Cantidad de CPU a usar en el ajuste
- **class_weight:** peso a las ctegorias.


In [ ]:
rf = RandomForestClassifier(n_estimators = 200)
rffit = rf.fit(Xtr, np.ravel(ytr))
preds_rf = rffit.predict(Xte)

In [ ]:
metrics.confusion_matrix(yte, preds_rf)

También es posible predecir las probabilidades

In [ ]:
preds_prob = rffit.predict_proba(Xte)
roc = metrics.roc_curve(yte, preds_prob[:, 1])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(roc[0], roc[1])
plt.plot([0,1], [0,1])
plt.show()

In [ ]:
metrics.auc(roc[0], roc[1])

#### Importancia Relativa

In [ ]:
rffit.feature_importances_

De aquí se desprende que **RevolvingUtilizationOfUnsecuredLines** es la variable que más afecta

### Ejemplo GBM

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

Los argumentos básicos son los siguientes

```GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, max_features=None, max_leaf_nodes=None, validation_fraction=0.1)```

- **loss** función de pérdida, solo tiene la devianza y una pérdida exponencial
- **learning_rate** Tasa de aprendizaje, es un parámetro de encogimiento de los nuevos modelos incorporados.
- **n_estimators** indica la cantidad de iteraciones del algoritmo, es decir cuantos nuevos modelos.
- **subsample** Cantidad de puntos al azar para ajustar el algoritmo, se usar para el SGBM
Notar que el resto de los argumentos hacen referencia a los árboles que va a ajustar el GBM

In [ ]:
gbm = GradientBoostingClassifier(n_estimators = 100)
gbmfit = gbm.fit(Xtr, np.ravel(ytr))
preds_gbm = gbmfit.predict(Xte)

In [ ]:
metrics.confusion_matrix(yte, preds_gbm)

Veamos la curva ROC

In [ ]:
preds_prob_gbm = gbmfit.predict_proba(Xte)
roc = metrics.roc_curve(yte, preds_prob_gbm[:, 1])

plt.plot(roc[0], roc[1])
plt.plot([0,1], [0,1])
plt.show()

In [ ]:
metrics.auc(roc[0], roc[1])